In [1]:
import os, glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch

from ultralytics import YOLO
import easyocr
import utils

In [2]:
if "temp" not in os.listdir():
    os.makedirs("./temp")

<h3 style="color:orange">Hyperparameters</h3>

In [3]:
DEVICE="cuda" if torch.cuda.is_available() else "cpu"


reader=easyocr.Reader(['en'],gpu=torch.cuda.is_available())

print(f"{DEVICE=}")

vid_dir=["./dataset/video/demo.mp4"]

vid=cv2.VideoCapture(vid_dir[0])

# video metadata
frame_count=int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
W=int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
H=int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS=int(vid.get(cv2.CAP_PROP_FPS))

print(f"frame count: {frame_count}\nframe width: {W}\nframe hight: {H}\nFPS: {FPS}")

# save video folder
out_mp4=cv2.VideoWriter("./temp/video/ans.mp4",cv2.VideoWriter_fourcc(*"mp4v"),FPS,(W,H))

DEVICE='cuda'
frame count: 631
frame width: 1114
frame hight: 720
FPS: 30


<h3 style="color:orange">Load Model</h3>

In [4]:
model_weight=[path for path in glob.glob("./Model/weights/*.pt")]
print(model_weight)
model=YOLO(model_weight[1]).to(DEVICE)

matter_tracker=utils.sort.Sort()   #use for tracting object in continues frames

print(model)

['./Model/weights\\yolov8n.pt', './Model/weights\\YOLOv8_LPD_largeset.pt', './Model/weights\\YOLOv8_LPD_roboflow.pt']
YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1,

<h3 style="color:orange">model infernce</h3>

In [ ]:
results={}

if "video" not in os.listdir("./temp/"): os.makedirs("./temp/video")
out_mp4=cv2.VideoWriter("./temp/video/ans.mp4",cv2.VideoWriter_fourcc(*"mp4v"),FPS,(W,H))

LIMIT=50

ret,frame_num=True,-1
while(ret):
    frame_num+=1
    ret,frame=vid.read()
    print(f"{ret}, Frame no. {frame_num}")
    if(ret):

        # if(frame_num>LIMIT): break     #debug
        
        results[frame_num]={}

        # detect numberplate
        detections=model(frame)[0]
        detection_list=[]
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=detection
            if(score>=0.5):
                # print([int(x1),int(y1),int(x2),int(y2),score])
                detection_list.append([int(x1),int(y1),int(x2),int(y2),score])
        print("\n")
    

        ## track vehicle 
        # track_id=matter_tracker.update(np.asarray(detection_list))
        # print(track_id)

        # crop Lincence plate
        corp_license_plate=frame[int(y1):int(y2),int(x1):int(x2),:]
        # print(corp_license_plate.shape)
        ## cv2.imwrite(f"./temp/{frame_num}.png",corp_license_plate)    ## DEBUG

        ## correct one
        # ans=utils.utils.read_license_plate(corp_license_plate)
        
        # wrting into video frame
        detect=reader.readtext(corp_license_plate)
        for x in detect:
            if(len(x)!=0):
                print(x)
                cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),(int(y2))),(0,255,255),2)

                if "ans" not in os.listdir("./temp/"): 
                    os.makedirs("./temp/ans")
                cv2.imwrite(f"./temp/ans/{frame_num}.png",frame)
        
        #  save video
        out_mp4.write(frame)

        continue

out_mp4.release()
cv2.destroyAllWindows()

True, Frame no. 0

0: 416x640 2 License Platess, 51.7ms
Speed: 2.0ms preprocess, 51.7ms inference, 122.5ms postprocess per image at shape (1, 3, 416, 640)


([[np.int32(28), np.int32(6)], [np.int32(167), np.int32(6)], [np.int32(167), np.int32(55)], [np.int32(28), np.int32(55)]], 'RIBF', np.float64(0.20223328471183777))
True, Frame no. 1

0: 416x640 2 License Platess, 7.1ms
Speed: 1.9ms preprocess, 7.1ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 640)


([[np.int32(25), np.int32(13)], [np.int32(77), np.int32(13)], [np.int32(77), np.int32(49)], [np.int32(25), np.int32(49)]], '60', np.float64(0.9994920839701239))
True, Frame no. 2

0: 416x640 2 License Platess, 7.6ms
Speed: 2.4ms preprocess, 7.6ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 640)


([[np.int32(23), np.int32(7)], [np.int32(79), np.int32(7)], [np.int32(79), np.int32(47)], [np.int32(23), np.int32(47)]], '60', np.float64(0.9992029275803506))
True, Frame no. 3

0: 416x640 3 License Platess, 7.1m